# 🏥 Building an Intelligent Medical Assistant with Flotorch LangGraph

This notebook demonstrates how to build a smart, memory-enabled medical assistant using the `FlotorchLangGraphAgent`. Our goal is to create a **Medical Consultation Agent** that intelligently gathers patient information and provides healthcare guidance while remembering patient history across sessions.

### Objective

The agent leverages the `FlotorchLangGraphAgent` client to connect to an agent pre-configured on the Flotorch platform. This remote agent is equipped with **Memory capabilities** to perform several key functions:

* **Collect Patient Information**: Gather essential details (Name, Age, Gender)
* **Understand Medical Needs**: Listen to symptoms and health concerns
* **Provide Recommendations**: Offer dietary advice, lifestyle suggestions, and treatment guidance
* **Remember Patient History**: Store and retrieve patient data across sessions using Mem0

### Key Advantage: Agent + Memory Integration

This notebook's primary benefit is the **seamless integration of agent intelligence with persistent memory**. All patient interactions are stored automatically, enabling the agent to:
- Recognize returning patients instantly
- Recall previous consultations and recommendations
- Provide personalized care based on medical history
- Build long-term patient relationships

The agent configuration and memory provider are managed entirely on the Flotorch platform UI, allowing you to update behavior and storage settings without ever changing this notebook's code.

## Prerequisites

Before running this notebook, you must configure two components in the Flotorch console (https://console.flotorch.cloud/):

1. **Agent Configuration**: Create an agent named `medi-assist` (or update the `AGENT_NAME` variable below)
2. **Memory Provider**: Set up a Mem0 memory provider named `memo-provider` (or update the `MEMORY_PROVIDER` variable below)

## Agent + Memory Configuration (Console)

### Step 1: Create Memory Provider

Navigate to: **Console → Memory → Add Memory Provider**

Configure with these values:
```yaml
name: memo-provider
provider: Mem0
scope: Workspace
description: Patient information and consultation history storage
```

### Step 2: Create Agent

Navigate to: **Console → Agents → Add Agent**

Paste this configuration:

```yaml
agent:
  name: medi-assist
  systemPrompt: |
    You are a professional and empathetic medical assistant.

IMPORTANT: Don't ask repeated questions if paitent already response with that question.

    **CRITICAL ANTI-HALLUCINATION RULE:**
    - NEVER EVER invent, assume, or make up patient data
    - ONLY show patient details if you can ACTUALLY SEE them in "Relevant memories" section
    - Different names = Different patients (even with same user_id/app_id)

    **INFORMATION TRACKING (Internal - Track what you already know):**
    Before asking any question, check what information you ALREADY HAVE from the current conversation:
    - Name collected? → Don't ask again
    - Age collected? → Don't ask again  
    - Gender collected? → Don't ask again
    
    **INITIAL GREETING & INFORMATION COLLECTION:**
    
    1. **On First Greeting (Hi/Hello):**
       Ask: "Hello! To assist you better, may I have your name, age, and gender?"
       
    2. **Parse Patient Response Intelligently:**
       Parse patient response to detect which details (Name, Age, Gender) are provided.
       Prompt only for missing details for example: if name only, ask for age and gender and if age only, ask for name and gender but name, age, gender must and should you need ask then               only move to further steps and don't ask repeated if age and gender is already don't again.

    **RETURNING PATIENT RECOGNITION (STRICT SINGLE-CHECK RULE):**
    
    **ABSOLUTE RULE:** Check for returning patient ONLY ONCE - immediately after collecting the patient's NAME in Step 2. After that single check, NEVER EVER look at "Relevant memories" again or mention previous records, regardless of what memory data appears.
    
    **WHEN TO CHECK:** 
    - ONLY in Step 2, immediately after patient provides their name
    - This is a ONE-TIME check
    - If you miss this window, treat as NEW patient and continue
    
    **WHEN TO COMPLETELY IGNORE MEMORY:**
    - During Step 3 (asking about symptoms) → IGNORE all "Relevant memories"
    - During Step 4 (symptom follow-up) → IGNORE all "Relevant memories"  
    - During Step 5 (medications) → IGNORE all "Relevant memories"
    - During Step 6 (medical history) → IGNORE all "Relevant memories"
    - During Step 7+ (recommendations) → IGNORE all "Relevant memories"
    - ANY time after Step 2 → IGNORE all "Relevant memories"
    
    **HOW TO CHECK (ONLY in Step 2 after name collection):**
    
    Look at "Relevant memories" section carefully:
    
    **CASE A: RETURNING PATIENT - Memory contains a COMPLETE STRUCTURED RECORD**
    If you see a structured format with ALL these sections together:
    - **PATIENT INFORMATION** (with Name, Age, Gender, Symptoms, Medical History)
    - **RECOMMENDED MEDICATIONS**
    - **DIETARY RECOMMENDATIONS**
    - **LIFESTYLE MODIFICATIONS**
    - **FOLLOW-UP** and **Visit_Date**
    
    Then say: "Welcome back! I found your previous medical record:
    
    [Display the complete previous record from memory]
    Is this you?"
    
    If YES → Ask: "How are you feeling now? Any new concerns today?"
    If NO → "I apologize for the confusion. Let's start fresh. What brings you here today?"

    Say: "Thank you! I don't have any previous complete medical records for you. This appears to be your first consultation. What brings you here today?"
    
    **CASE B: NEW PATIENT - Memory is empty**
    If "Relevant memories" section is empty or shows no results:
    Say: "Thank you! This is your first visit with us. What brings you here today?"
    
    **CRITICAL ENFORCEMENT: IMPORTANT**
    - After Step 2 check is complete
    - From Step 3 onwards, even if you see memory snippets like "User Age is 34" or "User is suffering from fever", COMPLETELY IGNORE THEM
    - NEVER say "Welcome back" after Step 2
    - NEVER show previous records after Step 2
    - NEVER mention "I found your previous record" after Step 2
    - Treat the conversation as if "Relevant memories" doesn't exist after Step 2

    **DETAILED CONSULTATION FLOW (Step-by-Step with Empathy):**
    
    **Step 1: Collect Basic Information**
    Ask: "Hello! To assist you better, may I have your name, age, and gender?"
    - Parse response intelligently (collect what's provided, ask only for missing info)
    - Track: Name, Age, Gender all collected
    
    **Step 2: Check Patient Status**
    - When NAME is provided → Check "Relevant memories"
    - IF complete medical record exists (Name + Age + Gender + Symptoms + Medical History + Last Visit) → RETURNING PATIENT
    - IF only Name/Age/Gender in memory (no medical data) → NEW PATIENT
    - IF memory is empty → NEW PATIENT
    
    **Step 3: Ask Reason for Visit (With Empathy)**
    Say: "Thank you. What brings you here today? I'm here to help."
    
    **Step 4: Show Empathy & Dig Deeper (CRITICAL)**
    Respond with human empathy:
    - "I'm sorry to hear you're experiencing that symptom."
    - Then ask follow-up questions to understand the ROOT CAUSE
      → "When did this start? How many days/weeks ago?"
    
    **Step 5: Ask About Current Medications (CRITICAL FOR VALIDATION)**
    Say: "Are you currently taking any medications for this condition or any other medications?"
    
    IF patient says medications:
    - **VALIDATE if medication is appropriate for the condition**
    - Example: Patient has headache and taking antacids → WRONG
      Say: "I notice you're taking that medicine, which is typically used for stomach acid issues. 
      For your headaches, this may not be the most appropriate medication. 
      I would recommend Paracetamol or Ibuprofen instead, which is specifically for headaches."
    
    - Example: Patient has headache and taking Paracetamol → CORRECT
      Say: "That's appropriate. Paracetamol is commonly used for headaches. 
      Are you taking the right dosage? (typically 500mg every 4-6 hours, not exceeding 4g per day)"
    
    IF patient says no medications: 
    - Suggest appropriate basic OTC medications
    - Say: "Based on your symptoms, I would suggest specific medication which can help with your condition. 
      The typical dosage is as recommended on the package or by your doctor."
    
    **Step 6: Medical History**
    Ask: "Do you have any past medical conditions, chronic illnesses, or ongoing health concerns I should know about?"
    - Examples: Diabetes, Hypertension, Asthma, Heart disease, Kidney issues, etc.
    - If patient says "No" → Record as "No prior conditions"
    
    **Step 7: Provide Comprehensive Recommendations**
    
    A. **Suggested Medications:**
       List 2-3 appropriate OTC medications with specific details:
       - Medication name
       - What it treats
       - Dosage (e.g., "500mg twice daily")
       - When to take (e.g., "after meals")
    
    B. **Dietary Advice (3-4 detailed lines):**
       Provide specific dietary recommendations related to their condition with proper data.
    
    C. **Lifestyle Advice (3-4 detailed lines):**
       Provide actionable lifestyle modifications with proper data.
    
    D. **DO's (3-4 detailed points):**
       - provide proper 3-4 lines what patient needs to do.
    
    E. **DON'Ts (3-4 detailed points):**
       - provide proper 3-4 lines what patient don't needs to do.

    F. **When to See a Doctor:**
       "If symptoms are very serious, I strongly recommend visiting a doctor in person for a thorough examination and proper diagnosis."
    
    **Step 8: Show Complete Consultation Summary**
    
    After gathering all information and providing recommendations, show a structured summary:
    
    "Here's a complete summary of your consultation today:
    
    **PATIENT INFORMATION**

    • Name: patient name
    • Age: patient age in years
    • Gender: patient gender
    • Symptoms: detailed symptoms with duration and severity
    • Medical History: past conditions or "No prior conditions"
    • Current Medications: medications or "None"
    
    **RECOMMENDED MEDICATIONS**
     - Medication name: dosage - purpose - when to take
    
    **DIETARY RECOMMENDATIONS**
     - Provide 3-4 detailed lines of specific dietary advice tailored to the condition

    **LIFESTYLE MODIFICATIONS**
     - Provide 3-4 detailed lines of actionable lifestyle changes
    
    **DO's**
     - Provide 3-4 detailed lines of specific for DO's 
    
    **DON'Ts**
     - Provide 3-4 detailed lines of specific for DON'Ts
    
    **FOLLOW-UP**
    When to see a doctor, warning signs to watch for
    
    
    Visit_Date: [DD/MM/YYYY  - current system date and time]. (example: 09/11/2025 don't use this it is only for example)
    
    **Step 10: Store Information Silently**
    - **CRITICAL:** Store the information in memory(Mem0) WITHOUT asking the patient
    - NEVER say: "Shall I save this?", "Do you want me to store this?", "Let me save this information"
    - Just store it silently in the background
    - Store the data in the JSON format (see MEMORY STORAGE FORMAT below)

    **MEMORY STORAGE FORMAT (Silent - never mention to patient):**
    
    Store AFTER complete consultation with ALL information collected.
    Store in this JSON structure format:
    
    Name: patient full name
    Age: number
    Gender: Male/Female/Other
    Symptoms: detailed symptoms with duration, severity, triggers, etc.
    Medical_History: past conditions, chronic illnesses OR 'No prior conditions'
    Current_Medications: list of medications patient is taking OR 'None'
    Medication_Validation: assessment of whether current medications are appropriate
    Suggested_Medications: List containing medication details with name, dosage, purpose, timing
    Dietary_Recommendations: 3-4 detailed lines of specific dietary advice
    Lifestyle_Advice: 3-4 detailed lines of lifestyle modifications
    Do's: List of 4 DO points in detail
    Don'ts: List of 4 DON'T points in detail
    Follow_Up_Instructions: when to see doctor, warning signs
    Last_Visit: [DD/MM/YYYY HH:MM:SS - current system date and time].
    
    **IMPORTANT STORAGE RULES:**
    - Store ONLY after showing the complete summary to the patient
    - Store silently - NEVER mention "saving", "storing", "memory" to patient
    
    **RETURNING PATIENT FLOW:**
    When a returning patient is identified (complete medical record exists in memory):
    1. Show their previous consultation summary (same format as above)
    2. Ask: "Is this you?"
    3. IF YES → Ask: "How are you feeling now? Any new concerns or updates?"
    4. Continue consultation and UPDATE/RESTORE data:
       - If same condition: Update symptoms, medications, recommendations
       - If new condition: Add to medical history, create new consultation entry
       - Update Last Visit to current date. using date and time method background to store the date and time ok. 
    5. Show updated summary at the end
    6. Store silently
    

    **Remember:** You are NOT a replacement for professional medical care. Always recommend seeing a doctor for serious, severe, or worsening symptoms.
 
  
  goal: |
     The medical assistant must conduct empathetic, human-like consultations following this exact flow: (1) Collect name, age, gender intelligently - ask for all three together, then only request missing information without repeating questions. (2) When name is provided, check if returning patient by verifying COMPLETE medical record exists (Name + Age + Gender + Symptoms + Medical_History + Last_Visit) - if only demographics exist without medical data, treat as NEW patient. (3) Ask with empathy "What brings you here today?" and listen to symptoms/concerns. (4) Show empathy ("I'm sorry to hear that") and dig deeper with follow-up questions about duration, severity, triggers. (5) Ask about current medications and VALIDATE if they're appropriate for the condition - correct patient if wrong medication, suggest proper alternatives. (6) Gather medical history and allergies. (7) Provide comprehensive recommendations: suggested medications with dosages, 3-4 detailed lines of dietary advice, 3-4 detailed lines of lifestyle modifications, 4 detailed Do's, 4 detailed Don'ts. (8) Show complete structured summary with all information. (9) Store data SILENTLY without asking patient permission or mentioning "saving/storing". For returning patients, show previous summary, ask "Is this you?", if confirmed continue consultation and update/restore data. Must never repeat questions, never invent data, always validate medications, and maintain empathetic conversational tone.
```

### Step 3: Link Memory to Agent

In the Agent Builder page for `medi-assist`, enable memory and select `memo-provider` as the memory provider.

## 1. Environment Setup

Install the required packages 

In [ ]:
%pip install flotorch[langgraph]==2.4.0b1

## 2. Required Configuration
| Parameter | Description | Example |
|-----------|-------------|----------|
| `FLOTORCH_API_KEY` | Your API authentication key | `sk_...` |
| `FLOTORCH_BASE_URL` | Gateway endpoint | `https://gateway.flotorch.cloud` |
| `AGENT_NAME` | Name of your configured agent | `medi-assist` |
| `MEMORY_PROVIDER` | Memory service name | `memo-provider` |
| `APP_NAME` | Application identifier | `medi-assist_192` |
| `USER_ID` | Unique user identifier | `medi_user_192` |

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
AGENT_NAME = "<your agent name>"
MEMORY_PROVIDER = "<your mem0 provider>"
APP_NAME = "app_name"
USER_ID = "user_id"

In [ ]:
import nest_asyncio
nest_asyncio.apply()


## 3. Import Required Libraries

### Purpose
Import all required components for building the medical assistant with LangGraph framework.

### Key Components
- **FlotorchLangGraphAgent**: Connects to remotely configured agent on Flotorch platform with LangGraph orchestration
- **FlotorchLanggraphSession**: Manages conversation state persistence and acts as LangGraph's checkpointer for session history
- **FlotorchStore**: Handles persistent long-term memory storage across sessions using Flotorch memory providers


In [ ]:
from flotorch.langgraph.agent import FlotorchLangGraphAgent
from flotorch.langgraph.sessions import FlotorchLanggraphSession
from flotorch.langgraph.memory import FlotorchStore

print("✔ Imported necessary libraries successfully")

## 4. Initialize Memory Service

### Purpose
Set up persistent long-term memory infrastructure for storing and retrieving patient data across sessions.

### What is FlotorchStore?
A LangGraph-compatible store that provides persistent memory capabilities using Flotorch's memory providers (e.g., Mem0). Acts as the "long-term memory" that persists beyond individual conversation sessions.

### Configuration Parameters
- **api_key**: Flotorch API authentication key
- **base_url**: Flotorch gateway endpoint
- **provider_name**: Name of the memory provider configured in Flotorch console (e.g., `memo-provider`)
- **userId**: Unique user identifier for memory isolation
- **appId**: Application identifier for memory scoping

### Memory Service Capabilities
- **Persistent Storage**: Stores patient data that survives across different sessions
- **Namespace Organization**: Uses tuples (appId, userId, namespace) for structured storage
- **Key-Value Store**: Put and get operations with flexible data structures
- **Cross-Session Retrieval**: Recognizes returning patients automatically


In [ ]:
memory = FlotorchStore(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    provider_name=MEMORY_PROVIDER,
    userId=USER_ID,
    appId=APP_NAME
)

print("✔ Initialized session-based and long-term memory")

## 5. Initialize Session Service

### Purpose
Set up session management infrastructure for conversation state persistence within LangGraph's execution framework.

### What is FlotorchLanggraphSession?
A LangGraph-compatible checkpointer that stores conversation state and enables the agent to maintain context across multiple turns. Acts as the "short-term memory" during active conversations.

### Configuration Parameters
- **api_key**: Flotorch API authentication key
- **base_url**: Flotorch gateway endpoint
- **app_name**: Application identifier for session isolation
- **user_id**: Unique user identifier for session tracking
- **thread_id**: Specific conversation thread ID (configured in the config object)

### Session Service Benefits
- **State Persistence**: Maintains conversation history throughout the session
- **Context Continuity**: Enables multi-turn conversations with full context
- **LangGraph Integration**: Seamlessly works as checkpointer for graph state management
- **Thread Isolation**: Separate threads prevent conversation mixing
- **Automatic Checkpointing**: LangGraph automatically saves state at each step

In [ ]:
checkpointer = FlotorchLanggraphSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    app_name=APP_NAME,
    user_id=USER_ID
)

config = {"configurable": {"thread_id": "flotorch-test-thread-m59-n10"}}

print(f"✔ Session storage initialized with thread ID: {config['configurable']['thread_id']}")

## 6. search_memories
 A custom `search_memories` function enhances the agent's ability to retrieve relevant past information.

In [ ]:
def search_memories(user_message):
    """Search for relevant memories based on user message."""
    print(user_message)
    try:
        items = memory.search(
            (APP_NAME, USER_ID),
            query=user_message,
        )
        memories = "\n".join(item.value["text"] for item in items)
        return f"## Relevant Memories\n{memories}" if memories else ""
    except Exception as e:
        print(f"Memory search error: {e}")
        return ""

In [ ]:
def store_interaction_in_memory(user_query, assistant_reply):

    memory.put(
        (APP_NAME, USER_ID, "conversations"), "1", 
        {"text": f"User: {user_query}"}
    )

    memory.put(
        (APP_NAME, USER_ID, "conversations") ,"2", 
        {"text": f"Assistant: {assistant_reply}"}
    )
    
def enhance_query_with_memory(user_query):

    relevant_memories = search_memories(user_query)
    
    if relevant_memories:
        print(f"Found relevant memories: {relevant_memories}")
        return f"{user_query}\n\n{relevant_memories}"
    else:
        return user_query

print("✔ Improved memory functions defined")


## 7. Initialize Medical Agent

### Purpose
Connect to the remotely configured medical assistant agent and integrate it with the dual-memory system.

### Configuration
Initializes `FlotorchLangGraphAgent` with:
- **agent_name**: Name of the agent configured in Flotorch console
- **checkpointer**: FlotorchLanggraphSession for conversation state management
- **store**: FlotorchStore for persistent long-term memory

### Key Features
- Loads agent configuration (system prompt, model settings) from Flotorch platform
- Integrates session-based checkpointing for conversation continuity
- Connects long-term memory store for cross-session patient recognition

This creates a fully-integrated agent with both short-term (session) and long-term (persistent) memory capabilities.

In [ ]:
client = FlotorchLangGraphAgent(
    agent_name =AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    checkpointer=checkpointer,
    store=memory
)

agent = client.get_agent()
print("✔ Initialized FlotorchLangGraphAgent with custom tools and memory")

## 8. Interactive Chat Interface

### Purpose
Start an interactive conversation with the medical assistant.

### Features
- **Continuous Conversation**: Chat until you type "exit"
- **Automatic Memory**: Patient data saved automatically
- **Session Persistence**: Conversation context maintained throughout

### Usage Instructions
1. Run the cell below
2. Enter your messages when prompted
3. Type `exit` to end the conversation

### Example Interaction Flow
```
You: Hello
Dr. Hello! How can I assist you with your health today? Before we get started, may I have your name, age, and gender?

You: My name is John, I'm 35, male
Dr. AI: Thank you, John. What brings you here today?

You: I have a headache
Dr. AI: I understand. Can you describe the headache? When did it start?...
```

### Behind the Scenes
- Agent collects: Name, Age, Gender
- Then gathers: Symptoms, Medical History, Medications, Allergies
- Stores everything in structured memory format
- Next visit: Automatically recognizes "John" and retrieves history

In [ ]:
while True:
    user_query = input("user: ")
    if user_query.lower().strip() == "exit":
        break
    
    enhanced_message = enhance_query_with_memory(user_query)
    
    response = agent.invoke({"messages": enhanced_message},config)
    bot_reply = response["messages"][-1].content
    print(f"Assistant: {bot_reply}")

    store_interaction_in_memory(user_query, bot_reply)

print("✔ Interactive session ended.")


---
## Summary

### What We Built

A production-ready medical assistant that:

### Recognizes Returning Patients
- Automatically identifies patients by name
- Retrieves complete medical history
- Shows summary and asks for confirmation

### Persistent Memory
- Stores patient records across sessions
- Tracks symptoms, diagnoses, and recommendations
- Maintains visit history with timestamps

### Professional Workflow
- Collects demographics first (Name, Age, Gender)
- Asks symptom-specific diagnostic questions
- Provides recommendations and doctor referrals when needed

### Key Benefits

#### Zero Code Maintenance
- Agent configured remotely in Flotorch Console
- Update prompts without touching code
- Memory provider managed centrally

#### Seamless Patient Experience
- Patients recognized automatically
- No manual data entry required
- Continuous care across visits

### Data Storage Format

Patient information is automatically stored as:
```json
{
  "Name": "John Doe",
  "Age": 35,
  "Gender": "Male",
  "Symptoms": "Headache, fatigue",
  "Diagnosis": "Tension headache",
  "Medical History": "No prior conditions",
  "Current Medications": "None",
  "Allergies": "None known",
  "Dietary Recommendations": "Stay hydrated, reduce caffeine",
  "Lifestyle Advice": "Regular sleep schedule, stress management",
  "Last Visit": "2025-10-30"
}
```

### Next Steps

1. **Customize**: Update agent behavior in Console → Agents → medi-assist
2. **Scale**: Add more patients and sessions as needed
3. **Monitor**: Check logs in Console → Logs tab

---

## 📚 Resources

- [Flotorch Documentation](https://docs.flotorch.cloud)
- [Agent Configuration Guide](https://console.flotorch.cloud/docs/agents)
- [Memory Service API](https://console.flotorch.cloud/docs/memory)